# Tutorial Sqlite3

Neste tutorial, você criará um banco de dados de filmes do Monty Python usando funcionalidades básicas do sqlite3. Pressupõe-se um entendimento fundamental de conceitos de banco de dados, incluindo cursores e transações.

Primeiramente, precisamos criar um novo banco de dados e abrir uma conexão com ele para permitir que o sqlite3 trabalhe com o banco. Use o comando `sqlite3.connect()` para criar uma conexão com o banco de dados chamado **tutorial.db** no diretório de trabalho atual, criando-o implicitamente caso ele não exista:

In [11]:
import sqlite3
con = sqlite3.connect("tutorial.db")

O objeto de conexão retornado, **con**, representa a conexão com o banco de dados armazenado em disco.

Para executar instruções SQL e obter os resultados das consultas SQL, será necessário usar um cursor de banco de dados. Use o comando `con.cursor()` para criar o cursor:

In [12]:
cur = con.cursor()

Agora que temos uma conexão com o banco de dados e um cursor, podemos criar uma tabela chamada **movie** com colunas para título, ano de lançamento e pontuação de avaliação. Para simplificar, podemos usar apenas os nomes das colunas na declaração da tabela – graças à funcionalidade de tipagem flexível do SQLite, especificar os tipos de dados é opcional. Execute a instrução **CREATE TABLE** chamando `cur.execute(...)`:

In [17]:
cur.execute("CREATE TABLE movie(title, year, score)")

Podemos verificar se a nova tabela foi criada consultando a tabela **sqlite_master**, integrada ao SQLite, que agora deve conter uma entrada para a definição da tabela **movie** (consulte *The Schema Table* para mais detalhes). Execute essa consulta chamando `cur.execute(...)`, atribua o resultado à variável **res** e use `res.fetchone()` para obter a linha resultante:

In [18]:
res = cur.execute("SELECT name FROM sqlite_master")
res.fetchone()

('movie',)

Podemos ver que a tabela foi criada, pois a consulta retorna uma tupla contendo o nome da tabela. Se consultarmos a **sqlite_master** para uma tabela inexistente, como **spam**, `res.fetchone()` retornará **None**:

In [19]:
res = cur.execute("SELECT name FROM sqlite_master WHERE name='spam'")
res.fetchone() is None

True

Agora, adicione duas linhas de dados fornecidos como literais SQL executando uma instrução **INSERT**, novamente utilizando `cur.execute(...)`:

In [20]:
cur.execute("""
    INSERT INTO movie VALUES
        ('Monty Python and the Holy Grail', 1975, 8.2),
        ('And Now for Something Completely Different', 1971, 7.5)
""")

A instrução **INSERT** abre implicitamente uma transação, que precisa ser confirmada antes que as alterações sejam salvas no banco de dados (consulte *Controle de Transações* para mais detalhes). Use `con.commit()` no objeto de conexão para confirmar a transação:

In [21]:
con.commit()

Podemos verificar se os dados foram inseridos corretamente executando uma consulta **SELECT**. Use o já conhecido `cur.execute(...)` para atribuir o resultado à variável **res** e chame `res.fetchall()` para retornar todas as linhas resultantes:

In [22]:
res = cur.execute("SELECT score FROM movie")
res.fetchall()

[(8.2,), (7.5,)]

O resultado é uma lista de duas tuplas, uma para cada linha, contendo o valor da coluna de pontuação correspondente.

Agora, insira mais três linhas chamando `cur.executemany(...)`:

In [ ]:
data = [
    ("Monty Python Live at the Hollywood Bowl", 1982, 7.9),
    ("Monty Python's The Meaning of Life", 1983, 7.5),
    ("Monty Python's Life of Brian", 1979, 8.0),
]
cur.executemany("INSERT INTO movie VALUES(?, ?, ?)", data)
con.commit()  # Lembre-se de confirmar a transação após executar o INSERT.

Note que os marcadores de posição **?** são usados para vincular os dados à consulta. Sempre use marcadores de posição em vez de formatação de strings para vincular valores do Python a instruções SQL, a fim de evitar ataques de *SQL injection* (consulte *Como usar marcadores de posição para vincular valores em consultas SQL* para mais detalhes).

Podemos verificar se as novas linhas foram inseridas executando uma consulta **SELECT**, desta vez iterando sobre os resultados da consulta:

In [24]:
for row in cur.execute("SELECT year, title FROM movie ORDER BY year"):
    print(row)

(1971, 'And Now for Something Completely Different')
(1975, 'Monty Python and the Holy Grail')
(1979, "Monty Python's Life of Brian")
(1982, 'Monty Python Live at the Hollywood Bowl')
(1983, "Monty Python's The Meaning of Life")


Cada linha é uma tupla de dois itens (ano, título), correspondendo às colunas selecionadas na consulta.

Por fim, verifique se o banco de dados foi gravado no disco chamando `con.close()` para fechar a conexão existente, abrindo uma nova conexão, criando um novo cursor e, em seguida, consultando o banco de dados:

In [ ]:
con.close()

new_con = sqlite3.connect("tutorial.db")
new_cur = new_con.cursor()

res = new_cur.execute("SELECT title, year FROM movie ORDER BY score DESC")
title, year = res.fetchone()

print(f'The highest scoring Monty Python movie is {title!r}, released in {year}')

new_con.close()

The highest scoring Monty Python movie is 'Monty Python and the Holy Grail', released in 1975
